# density

拖山的经度和纬度是(120.14,31.406)，兰山嘴的经度和纬度是(119.89,31.212)。我需要分析这2个点位逐日的叶绿素a浓度，请你修改下面的代码。如果受到云层影响导致数据缺失，请你用NAN代替。

创新点在于，将云层覆盖的区域设置为999999，这样在后续处理中可以很容易地识别出这些区域。

In [1]:
import aie

aie.Authenticate(token='ff0859af79711070b75b93848452399e')
aie.Initialize()

计算资源初始化中，请等待...
计算资源初始化完成.


In [2]:
pip install matplotlib numpy -i https://mirrors.aliyun.com/pypi/simple/

Looking in indexes: https://mirrors.aliyun.com/pypi/simple/
Note: you may need to restart the kernel to use updated packages.


In [ ]:
pip install geopandas -i https://mirrors.aliyun.com/pypi/simple/

In [2]:
import geopandas as gpd

In [4]:
# 读取shp文件
shapefile_path = '/root/Download/Modis-algae-aliyun/local-data/taihu.shp'
gdf = gpd.read_file(shapefile_path)

# 假设shp文件中只有一个几何对象，取第一个
geometry = gdf.geometry.iloc[0]
print(geometry)
# 输出((119.95082568376117 31.341888097180494, 119.98639104240964 31.388554715968827, ...

# 将geopandas的geometry对象转换为ee.Geometry
coords = list(geometry.exterior.coords)
flattened_coords = [coord for point in coords for coord in point]
taihu = aie.Geometry.Polygon(flattened_coords)
# 构造Polygon。输入参数支持GeoJSON 'Polygon'类型格式的坐标点集，
# 同时也支持包含偶数个数字的一维List，例如aie.Geometry.Polygon([aLng, aLat, bLng, bLat, ...])。
# 需注意，输入的坐标点将以EPSG:4326坐标系进行解析。

# 打印ee.Geometry对象以验证
print(taihu)

POLYGON ((119.95082568376117 31.341888097180494, 119.98639104240964 31.388554715968827, 120.00611934273043 31.40820067954559, 120.0213482061359 31.420557311454473, 120.02780893606551 31.42562749019076, 120.03023170978913 31.42966374717613, 120.0274051404449 31.43596389857637, 120.02025218945138 31.447873965243872, 120.00888822698595 31.45166321445573, 120.01067646473432 31.455403102339847, 120.01448368058568 31.46224277412878, 120.01806015608244 31.467556706624215, 120.02342486932757 31.473017934057427, 120.02827041677476 31.47729813281918, 120.02902032292732 31.479561147303205, 120.03525031250228 31.481725718642263, 120.03871141782174 31.48452974796941, 120.04223020822982 31.48275879185581, 120.04678733023371 31.479659537995836, 120.05734370145801 31.476461787529658, 120.06017027080222 31.47651098451827, 120.06380443138762 31.471541958062947, 120.06276609979179 31.468442332717288, 120.06345832085567 31.46578542930369, 120.06703479635243 31.4656870240255, 120.07078432711515 31.46052060

In [7]:
import datetime
import csv
import aie
import pandas as pd
import numpy as np
import chardet
from datetime import timedelta
from scipy.optimize import minimize

# 读取CSV文件并进行数据分析
csv_file = '/root/Download/Modis-algae-aliyun/aie-taihu/00-lanshanzui.csv'

# 检测文件编码
with open(csv_file, 'rb') as f:
    result = chardet.detect(f.read())

# 使用探测到的编码读取文件
df = pd.read_csv(csv_file, encoding=result['encoding'])

# 将日期列转换为日期时间格式
df['date'] = pd.to_datetime(df['date'])

# 设置日期为索引
df.set_index('date', inplace=True)

# 按日期分组并计算每日的统计数据
daily_stats = df['density'].resample('D').agg(['mean', 'std', 'count'])

# 计算95%置信区间
confidence_interval = 1.96 * daily_stats['std'] / np.sqrt(daily_stats['count'])

# 计算置信区间的上下限
daily_stats['lower_ci'] = daily_stats['mean'] - confidence_interval
daily_stats['upper_ci'] = daily_stats['mean'] + confidence_interval

start_date = datetime.date(2022, 7, 1)
end_date = datetime.date(2022, 7, 31)
delta = datetime.timedelta(days=1)

# 计算湖泊总面积
lake_area = taihu.area()

# 创建CSV文件并写入表头
with open('taihu_data.csv', 'w', newline='') as csvfile:
    csvwriter = csv.writer(csvfile)
    csvwriter.writerow(['日期', '站点', '藻密度', '北京时间'])

# 定义优化函数
def optimize_algae_density(params, ndci, target_mean, target_std):
    a, b, c = params
    predicted_density = a * ndci + b
    predicted_mean = np.mean(predicted_density)
    predicted_std = np.std(predicted_density)
    return (predicted_mean - target_mean)**2 + (predicted_std - target_std)**2

while start_date <= end_date:
    date_str = start_date.strftime("%Y_%m_%d")
    img = aie.Image(f'MODIS_MOD09GA_061_{date_str}')\
             .select(['sur_refl_b01', 'sur_refl_b04', 'sur_refl_b03','state_1km'])
    
    def mask_clouds(image):
        qa = image.select('state_1km')
        cloud_bit_mask = 1 << 10
        mask = qa.bitwiseAnd(aie.Image.constant(cloud_bit_mask))
        return mask
    
    def calculate_algae_density(image, cloud_mask, params):
        b01 = image.select('sur_refl_b01').toFloat()
        b03 = image.select('sur_refl_b03').toFloat()
        b04 = image.select('sur_refl_b04').toFloat()
        
        ndci = b04.subtract(b03).divide(b04.add(b03))
        
        # 使用优化后的参数计算藻密度
        a, b, c = params
        algae_density = ndci.multiply(aie.Image.constant(a)).add(aie.Image.constant(b)).rename(['AlgaeDensity'])
        
        # 将云层覆盖区域的藻密度值设置为极大值（例如9999）
        algae_density_masked = algae_density.where(cloud_mask.Not(), aie.Image.constant(9999))
        
        return image.addBands(algae_density_masked).select(['AlgaeDensity']).toFloat()
    
    # 应用云掩膜和计算藻密度
    cloud_mask = mask_clouds(img)
    
    # 获取当前日期的统计数据
    current_date = pd.to_datetime(start_date)
    if current_date in daily_stats.index:
        target_mean = daily_stats.loc[current_date, 'mean']
        target_std = daily_stats.loc[current_date, 'std']
    else:
        # 如果当前日期不在统计数据中，使用整体的平均值和标准差
        target_mean = daily_stats['mean'].mean()
        target_std = daily_stats['std'].mean()
    
    # 获取NDCI值
    ndci_values = img.normalizedDifference(['sur_refl_b04', 'sur_refl_b03']).sample(taihu, 100).aggregate_array('nd').getInfo()
    
    # 优化参数
    initial_params = [1000, 0, 0]  # 初始猜测值
    result = minimize(optimize_algae_density, initial_params, args=(ndci_values, target_mean, target_std))
    optimized_params = result.x
    
    processed_img = calculate_algae_density(img, cloud_mask, optimized_params)
    processed_img = processed_img.clip(taihu)
    
    # 指定采样区域，并在100分辨率下获取20个样本点，同时保留Geometry
    roi = aie.Geometry.BBox(119.89, 31.212, 119.90, 31.222)
    fcPointSamp = processed_img.sample(roi, 100, 1, 0, True)
    
    print(f"日期: {date_str}")
    print("藻密度:")
    print(fcPointSamp.aggregate_array('AlgaeDensity').getInfo())
    print("优化后的参数:", optimized_params)
    print("---")

    start_date += delta

日期: 2022_07_01
藻密度:


KeyboardInterrupt: 